1) Installazione di **Selenium**

In [ ]:
!pip install selenium

2. Importazione **librerie** necessarie (*BeautifulSoup*)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

3. Definisco una funzione che permetta di fare **scroll down** automatizzato della pagina web

In [ ]:
def scroll_down(driver):
    """Scroll down to bottom of the page"""
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(3)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

4. Inizializzo il **WebDriver** e lo mando alla pagina desiderata

In [ ]:
driver = driver = webdriver.Chrome()
Carrefour_home = 'https://www.carrefour.it/spesa-online/'
driver.get(Carrefour_home)

4. Raccolgo gli **url** che portano alle pagine delle varie *categorie*

In [ ]:
def extract_category(link):
    match = re.search(r'^((?:[^/]*/){4})([^/]*)', link)
    if match:
        return match.group(2)
    return None

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
categories_tags = soup.find_all('li', class_ = 'card glide__slide')

categories_links = []

for i in categories_tags:
    link = i.find('a').get('href')
    categories_links.append(link)
    category = i.find('span').get_text()
    categories.append(category)
   
categories_links = ['https://www.carrefour.it' + item for item in categories_links if 'www.carrefour' not in item]
categories_links = list(set(categories_links))

categories = [extract_category(i) for i in categories_links]
categories = [i.replace('-', ' ') for i in categories]

In [ ]:
for i in categories_links:
    print(i)

https://www.carrefour.it/spesa-online/articoli-per-la-casa/
https://www.carrefour.it/elettronica-casa-e-giocattoli/elettrodomestici-cucina/
https://www.carrefour.it/spesa-online/prodotti-biologici/
https://www.carrefour.it/spesa-online/pasta-riso-e-farina/
https://www.carrefour.it/elettronica-casa-e-giocattoli/telefonia/
https://www.carrefour.it/elettronica-casa-e-giocattoli/giocattoli-da-esterno/
https://www.carrefour.it/spesa-online/prodotti-prima-infanzia/
https://www.carrefour.it/elettronica-casa-e-giocattoli/tv-e-accessori/
https://www.carrefour.it/spesa-online/pane-e-snack-salati/
https://www.carrefour.it/spesa-online/birra-e-liquori/
https://www.carrefour.it/elettronica-casa-e-giocattoli/biciclette/
https://www.carrefour.it/spesa-online/vino/
https://www.carrefour.it/spesa-online/gelati-e-surgelati/
https://www.carrefour.it/spesa-online/condimenti-e-conserve/
https://www.carrefour.it/spesa-online/cura-della-casa/
https://www.carrefour.it/elettronica-casa-e-giocattoli/elettrodome

In [ ]:
for i in categories:
    print(i)

articoli per la casa
elettrodomestici cucina
prodotti biologici
pasta riso e farina
telefonia
giocattoli da esterno
prodotti prima infanzia
tv e accessori
pane e snack salati
birra e liquori
biciclette
vino
gelati e surgelati
condimenti e conserve
cura della casa
elettrodomestici pulizia casa stiro
giocattoli e videogiochi
frutta e verdura
cura della persona
carne
grandi elettrodomestici
acqua e analcolici
dolci e prima colazione
pesce
valigie
animali
climatizzazione e riscaldamento
salumi e formaggi
uova latte e latticini
computer e accessori
gastronomia
fai da te
console e accessori
casa


5. Creo la **funzione** che raccolga i *prodotti* dalle varie pagine

In [ ]:
def find_products(soup):
    prods = soup.find_all('h3', class_ = 'tile-description')
    
    products = []
    
    for tag in prods:
        product = tag.get_text()
        products.append(product)
    
    return products

6. Creo la **funzione** che raccoglie i *prezzi* dei vari prodotti

Se i prodotti sono in offerta, nell'html appare un nuovo tag analogo a quello del prezzo originario ma con quello scontato e dicitura *del*. Questo fa si che per ogni prodotto scontato vengano raccolti due prezzi invece che uno. In questo caso voglio raccogliere quello non scontato per avere un dato sempre valido.

In [ ]:
def find_prices_not_discounted(soup):
    all_prices = soup.find_all('span', class_ = 'unit-price')
    
    filtered_prices = []
    
    for tag in all_prices:
        if tag.find_parent(class_='sales discounted') is None:
            price = tag.get_text()
            filtered_prices.append(price)
    
    return filtered_prices

Nel sito del Carrefour, se il prodotto in questione non è disponibile il tag con il prezzo viene omesso e ce ne è uno diverso con dicitura *null*.  Qualora questo evento si verifichi...

7. Creo una **funzione** che prende i prodotti con dicitura *null* al posto del prezzo, per poterli poi droppare dalla lista di tutti i prodotti, se presenti

In [ ]:
def extract_not_available_product(soup):
    span = soup.find_all('span', class_='value', content='false')
    
    na_prods = []
    
    for i in span:
        a = i.parent.parent.parent.parent.parent
        b = a.find('h3', class_ = 'tile-description')
        c = b.get_text()
        na_prods.append(c)
    

8. Vado iterativamente nella pagina di ciascuna **categoria** e raccolgo **prezzi** e **prodotti**, facendoci una **lista**. Creo lista con prodotti non disponibili per fronteggiare eventuali problemi nel caso in cui la lista dei prezzi e la lista dei prodotti abbiano lunghezze diverse.

Stampo la lunghezza delle liste e l'eventuale prodotto/i non disponibile/i per tener traccia di ciò che fa il codice

In [ ]:
all_products = []
all_prices = []
not_availables = []
categories_column = []

In [ ]:
for i in range(len(categories_links)):
    link = categories_links[i]
    category = categories[i]
    driver.get(link)
    scroll_down(driver)
    scroll_down(driver)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    cat_products = find_products(soup)
    cat_prices = find_prices_not_discounted(soup)
    cat_not_available = extract_not_available_product(soup)
    cat = [category]*len(cat_products)
    
    print(category, len(cat_products), len(cat_prices), len(cat))
    
    for i in cat_products:
        all_products.append(i)
        
    for i in cat:
        categories_column.append(i)
        
    for i in cat_prices:
        all_prices.append(i)
        
    if cat_not_available:
        for i in cat_not_available:
            not_availables.append(i)
            print(i)    

articoli per la casa 907 907 907
elettrodomestici cucina 44 44 44
prodotti biologici 284 284 284
pasta riso e farina 637 637 637
telefonia 108 108 108
giocattoli da esterno 50 50 50
prodotti prima infanzia 248 248 248
tv e accessori 65 65 65
pane e snack salati 506 506 506
birra e liquori 259 259 259
biciclette 45 45 45
vino 594 594 594
gelati e surgelati 599 599 599
condimenti e conserve 887 887 887
cura della casa 1026 1026 1026
elettrodomestici pulizia casa stiro 31 31 31
giocattoli e videogiochi 256 256 256
frutta e verdura 521 521 521
cura della persona 1392 1392 1392
carne 151 151 151
grandi elettrodomestici 25 25 25
acqua e analcolici 442 442 442
dolci e prima colazione 199 199 199
pesce 142 142 142
valigie 17 17 17
animali 507 507 507
climatizzazione e riscaldamento 27 27 27
salumi e formaggi 666 666 666
uova latte e latticini 449 449 449
computer e accessori 38 38 38
gastronomia 298 298 298
fai da te 8 8 8
console e accessori 14 14 14
casa 285 285 285


9. **Salvo** le liste come csv per sicurezza

In [ ]:
import pandas as pd

CarrefourDataset = pd.DataFrame()
CarrefourDataset['Product'] = all_products
CarrefourDataset['unit Price'] = all_prices
CarrefourDataset['Category'] = categories_column

In [ ]:
CarrefourDataset.head()

,Product,unit Price,Category
0,Carrefour Correttore a nastro maxi 5x12 mm,"€ 1,89 al\npz/1.0 pz",articoli per la casa
1,Carrefour Confezione di 4 Penne a Sfera Blu Pe...,"€ 1,49 al\npz/1.0 pz",articoli per la casa
2,Carrefour Forbici punte arrotondate 17cm,"€ 2,99 al\npz/1.0 pz",articoli per la casa
3,Carrefour Confezione di 4 Penne a Sfera Nero P...,"€ 1,49 al\npz/1.0 pz",articoli per la casa
4,Carrefour Quaderno A4 punto metallico con cope...,"€ 1,59 al\npz/1.0 pz",articoli per la casa


In [ ]:
CarrefourDataset.to_csv('/Users/Chicca/Desktop/CarrefourDatasetScraped.csv')